In [1]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

In [2]:
simulation_start = 0
env = simpy.Environment(initial_time=simulation_start)
registry = {}

a = model.BasicActivity(
    ID= "a",
    env= env,
    name= "a",
    registry= registry,
    duration= 3,
)
a2 = model.BasicActivity(
    ID= "a2",
    env= env,
    name= "a2",
    registry= registry,
    duration= 2,
)
a3 = model.BasicActivity(
    ID= "a3",
    env= env,
    name= "a3",
    registry= registry,
    duration= 5,
)

c = model.BasicActivity(
    ID= "c",
    env= env,
    name= "c",
    registry= registry,
    duration= 1,
)

Pa=model.ParallelActivity(    
    env= env,
    name= "Pa",
    ID= "Pa",
    registry= registry,
    sub_processes= [a, a2,a3],
)

Sc=model.SequentialActivity(    
    env= env,
    name= "Sc",
    ID= "Sc",
    registry= registry,
    sub_processes= [c, Pa],
)

b = model.BasicActivity(
    ID= "b",
    env= env,
    name= "b",
    registry= registry,
    duration= 10,
    start_event=[
        {
            "name": "a",
            "type": "activity",
            "state": "done"
        }
    ]

)

Sb=model.SequentialActivity(    
    env= env,
    name= "Sb",
    ID= "Sb",
    registry= registry,
    sub_processes= [b],
)

In [3]:
model.register_processes([Sc,Sb])
env.run()

In [4]:
plot.vessel_planning([Sc,c,Pa,a,a2,a3,b])

In [5]:
pd.DataFrame(Pa.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:01,Pa,START,{},{}
1,1970-01-01 00:00:01,Pa,START,{},"{'type': 'subprocess', 'ref': 'a'}"
2,1970-01-01 00:00:01,Pa,START,{},"{'type': 'subprocess', 'ref': 'a2'}"
3,1970-01-01 00:00:01,Pa,START,{},"{'type': 'subprocess', 'ref': 'a3'}"
4,1970-01-01 00:00:03,Pa,STOP,{},"{'type': 'subprocess', 'ref': 'a2'}"


In [6]:
pd.DataFrame(Sc.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,Sc,START,{},{}
1,1970-01-01 00:00:00,Sc,START,{},"{'type': 'subprocess', 'ref': 'c'}"
2,1970-01-01 00:00:01,Sc,STOP,{},"{'type': 'subprocess', 'ref': 'c'}"
3,1970-01-01 00:00:01,Sc,START,{},"{'type': 'subprocess', 'ref': 'Pa'}"


In [7]:
pd.DataFrame(c.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,c,START,{},{}
1,1970-01-01 00:00:01,c,STOP,{},{}


In [8]:
pd.DataFrame(Sb.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,Sb,START,{},{}
1,1970-01-01 00:00:00,Sb,START,{},"{'type': 'subprocess', 'ref': 'b'}"
2,1970-01-01 00:00:14,Sb,STOP,{},"{'type': 'subprocess', 'ref': 'b'}"
3,1970-01-01 00:00:14,Sb,STOP,{},{}


In [9]:
pd.DataFrame(b.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,b,WAIT_START,{},{}
1,1970-01-01 00:00:04,b,WAIT_STOP,{},{}
2,1970-01-01 00:00:04,b,START,{},{}
3,1970-01-01 00:00:14,b,STOP,{},{}
